In [768]:
import pandas as pd
import time
import requests
import numpy as np
from datetime import datetime
from dateutil import parser
from pprint import pprint
import rdflib

In [677]:
date_of_birth = datetime.strptime('1969-00-01'[:4], "%Y")
# date_of_birth = datetime.strptime(wikidata_dict['date of birth (P569)'].split("T")[0], '+%Y-00-00')

In [683]:
"+1968"[1:5]

'1968'

In [679]:
date_of_birth.month

1

In [827]:
all_authors = pd.read_csv("authors.csv", index_col=0)
all_authors['info'] = "nothing"

In [828]:
def get_authors_affiliation_link(author_id):
    try:
        r = requests.get(f'http://ma-graph.org/entity/{author_id}')
        graph = rdflib.Graph()
        graph.parse(data=r.text)
        for s, p, o in graph:
            if "memberOf" in p:
                return o.toPython()
    except:
        return "Unknown"

In [829]:
# links = [get_authors_affiliation_link(i) for i in all_authors.id]

In [780]:
grid = pd.read_csv("grid.csv")

In [789]:
def get_affiliation_info(affiliation_link, grid_data=grid):
    try:
        r_aff = requests.get(affiliation_link)
        graph = rdflib.Graph()
        graph.parse(data=r_aff.text)
        infos = {}
        for s, p, o in graph:
            if "name" in p:
                infos["name"] = o.toPython()
            elif "homepage" in p:
                infos["homepage"] = o.toPython()
            elif "seeAlso" in p:
                infos["additionalWebsite"] = o.toPython()
            elif "grid" in p:
                infos["gridPage"] = o.toPython()
                grid_id = o.toPython().split("/")[-1]
                infos["city"] = grid_data[grid_data.ID == grid_id]["City"].values[0]
                infos["country"] = grid_data[grid_data.ID == grid_id]["Country"].values[0]

        return infos
    except:
        return {}

In [ ]:
all_affiliation_infos = [get_affiliation_info(link) if link != "Unknown" else {} for link in links]

In [839]:
all_affiliation_infos[-11]

{'additionalWebsite': 'http://en.wikipedia.org/wiki/Eötvös_Loránd_University',
 'city': 'Budapest',
 'country': 'Hungary',
 'gridPage': 'http://www.grid.ac/institutes/grid.5591.8',
 'homepage': 'http://www.elte.hu/en',
 'name': 'E?tv?s Lor?nd University'}

In [838]:
all_authors.iloc[-11]

id                                                    287580938
n_citation                                                   19
n_pubs                                                        2
name                                              Péter Aszalós
pubs          [{'i': '2001101397', 'r': 3}, {'i': '194468758...
tags          [{'t': 'Computer simulation'}, {'t': 'Degradat...
info                                                    nothing
Name: 67797, dtype: object

In [841]:
all_authors["info"] = all_affiliation_infos

In [843]:
all_authors.to_csv("authors_with_affiliation_info.csv")

In [844]:
! gsutil cp authors_with_affiliation_info.csv gs://noobs-ml/sem-search/CS_data/arxiv_data



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying file://authors_with_affiliation_info.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/253.5 MiB.                                    


In [879]:
sam = all_authors.sample()
print(sam.name.values[0])
print(sam.id.values[0])
print(sam["info"].values[0])

Pekka Korhonen
2028096276
{'homepage': 'http://www.aalto.fi/en/', 'name': 'Aalto University', 'additionalWebsite': 'http://en.wikipedia.org/wiki/Aalto_University', 'gridPage': 'http://www.grid.ac/institutes/grid.5373.2', 'city': 'Helsinki', 'country': 'Finland'}


In [880]:
with open('all_author_infos.pickle', 'rb') as handle:
    wikidata_infos = pickle.load(handle)

In [960]:
len([w for w in wikidata_infos if w])

28574

In [953]:
wiculled[917]

{'academic_degree': '',
 'age': None,
 'alive': False,
 'awards': ['doctor honoris causa',
  'Steele Prize for Mathematical Exposition'],
 'educated_at': ['Duke University'],
 'employers': ['University of Wisconsin–Madison'],
 'google_scholar_link': '',
 'notable_works': [''],
 'occupations': ['mathematician', 'university teacher'],
 'personal_websites': [],
 'wikidata_id': 'Q470503'}

In [961]:
def cull_data(w):
    if w != {}:
        if w["occupations"] != ['']:
            for i in w["occupations"]:
                if any(occ in i for occ in {"engineer", "linguist", "mathematician", "statistician", "computer", "researcher"}):
                    return w
            return {}
        else:
            return w
    else:
        return w
    


In [962]:
wiculled = [cull_data(w) for w in wikidata_infos]

In [963]:
len([w for w in wiculled if w != {}])

21685

In [964]:
all_authors["wikidata"] = wiculled

In [965]:
all_authors.to_csv("authors_with_wikidata.csv")

In [ ]:
s = time.time()
get_affiliation_info("http://ma-graph.org/entity/131827901")
print(time.time() - s)

In [929]:
all_authors[all_authors.name.str.contains("Anil K. J")].wikidata.values[0]

{'academic_degree': '',
 'age': None,
 'alive': True,
 'awards': ['W. Wallace McDowell Award',
  'ACM Fellow',
  'Fellow of the International Association for Pattern Recognition',
  'IEEE Fellow',
  'Fellow of the SPIE',
  'AAAS Fellow'],
 'educated_at': ['Ohio State University',
  'Indian Institute of Technology Kanpur'],
 'employers': ['Michigan State University'],
 'google_scholar_link': 'https://scholar.google.com/citations?user=g-_ZXGsAAAAJ',
 'notable_works': [''],
 'occupations': ['computer scientist', 'engineer'],
 'personal_websites': [],
 'wikidata_id': 'Q15783490'}

In [274]:
all_names = all_authors.name.tolist()

In [707]:
# RUN THIS EVENING!
all_author_infos = [get_author_info_v2(name) for name in all_names[:10]]

Q88060589
Q58312925


In [710]:
import pickle

with open('filename.pickle', 'wb') as handle:
    pickle.dump(all_author_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [711]:
with open('filename.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [605]:
all_names[:25]

['Florent Benaych-Georges',
 'Frederique Crete',
 'Maryam Karimzadehgan',
 'Fabio Massimo Zanzotto',
 'Stefano Serra-Capizzano',
 'Marco Cerami',
 'Anurat Chapanond',
 'Ajo Fod',
 'Gilles Degottex',
 'Erik Bylow',
 'Pierre Bonami',
 'Daniel Stutzbach',
 'Per Martin-Löf',
 'Slim Essid',
 'Vincent Perlbarg',
 'Hedi Tabia',
 'Koji Nonobe',
 'Sonja Zehetmayer',
 'Alexandrina Orzan',
 'Christian Goerick',
 'Yury Ustinovskiy',
 'Felipe Bravo-Marquez',
 'Vasileios Lampos',
 'Eiichiro Sumita',
 'Yasser Ganjisaffar']

In [629]:
get_author_info_v2(all_names[22])

{'academic_degree': [],
 'age': None,
 'alive': True,
 'awards': [],
 'educated_at': [],
 'employers': ['University College London'],
 'google_scholar_link': '',
 'notable_works': [],
 'occupations': ['researcher'],
 'personal_websites': 'https://www.lampos.net',
 'wikidata_id': 'Q30507137'}

In [256]:
def populate_authors_df_with_infos(infos):
    for index, info in enumerate(infos):
        all_authors.at[index, 'info'] = all_author_infos[index]

In [257]:
populate_authors_df_with_infos(all_author_infos)

In [63]:
import wptools
from wikidata.client import Client
from datetime import date

wikidata_client = Client()

In [175]:
# def get_author_wikidata_id(author_name):
#     page = wptools.page(author_name, silent=True)
#     page.get_query()
#     return page.data['wikibase']

In [198]:
def get_author_wikidata_id_new(name):
    r = requests.get(f"https://www.wikidata.org/w/api.php?action=wbsearchentities&language=en&search={name}&format=json")
    results = r.json()['search'][0]
    return results['id']

In [60]:
get_author_wikidata_id("Maarten de Rijke")

'Q6721195'

In [88]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [477]:
def get_info_from_wikidata(wd_id):
    entity = wikidata_client.get(wd_id, load=False)
    
    description = entity.description
    image = get_image(entity)
    dob, age = get_dob_and_age(entity)
    dod, alive = get_dod_and_alive(entity)
    webpage = get_webpage(entity)
    occupation = get_occupation(entity)
    employer = get_employer(entity)
    google_scholar = get_scholar(entity)
    
    return {"description": str(description), "wikidata_id": wd_id, "image": image, "date_of_birth": dob, "google_scholar_page": google_scholar,
            "age": age, "alive": alive, "occupation": occupation, "employer": employer, "personal_webpage": webpage}

In [478]:
def get_author_info(author_name):
    try:
        wiki_id = get_author_wikidata_id_new(author_name)
        return get_info_from_wikidata(wiki_id)
    except LookupError:
        return {}

In [487]:
s = time.time()
print(get_author_info("David Silver"))
print(time.time() - s)

{'description': 'AI researcher', 'wikidata_id': 'Q25208036', 'image': None, 'date_of_birth': None, 'google_scholar_page': 'https://scholar.google.com/citations?user=-8DNE4UAAAAJ', 'age': None, 'alive': True, 'occupation': 'Computer scientist', 'employer': 'Deepmind', 'personal_webpage': 'https://www.davidsilver.uk/'}
1.9209039211273193


In [492]:
s = time.time()
k = wptools.page(wikibase='Q937', silent=True).get()
pprint(k.data['wikidata'])
print(time.time() - s)

{'Academic Tree ID (P2381)': '3611',
 'Alexander Turnbull Library ID (P6683)': '120272',
 'AlloCiné person ID (P1266)': '36179',
 'Amazon author ID (P4862)': 'B000AP7JOU',
 'American Film Institute person ID (P5340)': '9130',
 'American National Biography ID (P4823)': '1300477',
 'Angelicum ID (P5731)': '86877',
 'Aozora Bunko author ID (P7311)': '1428',
 'Arnet Miner author ID (P5776)': '53f5b414dabfae472ef80501',
 'Atomic Heritage Foundation ID (P4590)': 'albert-einstein',
 'BDEL ID (P6231)': '80919',
 'BFI Films, TV and people ID (P4438)': '4ce2bb404fc06',
 'BHCL ID (P6656)': '31723',
 'BIU Santé person ID (P5375)': '5923',
 'Babelio author ID (P3630)': '2827',
 'BanQ author ID (P3280)': '0000007639',
 'Basque Vikidia ID (P7832)': 'Albert_Einstein',
 'BiblioNet author ID (P2188)': '157',
 'Biblioteca Nacional de España ID (P950)': 'XX834035',
 'Bibliothèque nationale de France ID (P268)': '119016075',
 'Biographie nationale de Belgique ID (P6234)': 'albert-einstein',
 'Bitraga autho

In [944]:
def parse_wikidata_v2(author_id):
    
    print(author_id)
    
    def clean_value_from_qcode(value):
        if value:
            return " ".join(value.split(" ")[:-1])
        else:
            return ""
    
    k = wptools.page(wikibase=author_id, silent=True).get_wikidata()
    wikidata_dict = k.data['wikidata']
    
    def try_value(value):
        try:
            val = wikidata_dict[value]
            if val:
                return val
            else:
                return []
        except:
            return []
    
    wikidata_id = author_id
    # These two fields are prone to mismatch so we leave them out for now
#     description = k.data["description"]
#     image_url = k.data['image'][0]['url']
    scholar = try_value('Google Scholar author ID (P1960)')
    if scholar:
        scholar_link = f"https://scholar.google.com/citations?user={scholar}"
    else:
        scholar_link = ""
    try:
        occupations = [clean_value_from_qcode(value) for value in [try_value('occupation (P106)')]]
    except AttributeError:
        occupations = [clean_value_from_qcode(value) for value in try_value('occupation (P106)')]
    websites = try_value('official website (P856)')
    try:
        employers = [clean_value_from_qcode(employer) for employer in [try_value('employer (P108)')]]
    except AttributeError:
        employers = [clean_value_from_qcode(employer) for employer in try_value('employer (P108)')]
    try:
        educated_at = [clean_value_from_qcode(ed) for ed in [try_value('educated at (P69)')]]
    except AttributeError:
        educated_at = [clean_value_from_qcode(ed) for ed in try_value('educated at (P69)')]
    try:
        if 'date of birth (P569)' in wikidata_dict and wikidata_dict['date of birth (P569)'] != '+2000-00-00T00:00:00Z':
            try:
                dob = wikidata_dict['date of birth (P569)']
#                 print(dob)
                if isinstance(dob, list):
                    if len(dob) > 1:
                        dob = dob[0]
#                 print("dob erna:", dob)
                date_of_birth = datetime.strptime(dob.split("T")[0], '+%Y-%m-%d')
            except ValueError:
                date_of_birth = datetime.strptime(dob.split("T")[0][1:5], "%Y")
            age = calculate_age(date_of_birth)
        else:
            age = None
    except:
        age = None
    try:
        notable_works = [clean_value_from_qcode(val) for val in [try_value('notable work (P800)')]]
    except AttributeError:
        notable_works = [clean_value_from_qcode(val) for val in try_value('notable work (P800)')]
        
    awards = try_value('award received (P166)')
    if awards and isinstance(awards, list):
        awards = [v for v in [clean_value_from_qcode(val) for val in awards] if v != ""]
    elif isinstance(awards, str):
        awards = [v for v in [clean_value_from_qcode(awards)] if v!= ""]
        
#     try:
#         awards = [clean_value_from_qcode(val) for val in [try_value('award received (P166)')]]
#     except AttributeError:
#         awards = [clean_value_from_qcode(val) for val in try_value('award received (P166)')]
    if 'date of death (P570)' in wikidata_dict:
        alive = False
    else:
        alive = True
    try:
        academic_degree = clean_value_from_qcode(try_value('academic degree (P512)'))
    except:
        academic_degree = try_value('academic degree (P512)')
    
    return {"wikidata_id": wikidata_id, "google_scholar_link": scholar_link, "occupations": occupations, "personal_websites": websites, 
           "employers": employers, "educated_at": educated_at, "age": age, "notable_works": notable_works, "awards": awards, 
            "academic_degree": academic_degree, "alive": alive}

In [954]:
get_author_info_v2('Q470503')

Q470503


{'academic_degree': '',
 'age': 99,
 'alive': False,
 'awards': ['doctor honoris causa',
  'Steele Prize for Mathematical Exposition'],
 'educated_at': ['Duke University'],
 'employers': ['University of Wisconsin–Madison'],
 'google_scholar_link': '',
 'notable_works': [''],
 'occupations': ['mathematician', 'university teacher'],
 'personal_websites': [],
 'wikidata_id': 'Q470503'}

In [959]:
[w for w in wiculled if w != {} and w['wikidata_id'] == 'Q470503']

[{'academic_degree': '',
  'age': None,
  'alive': False,
  'awards': ['doctor honoris causa',
   'Steele Prize for Mathematical Exposition'],
  'educated_at': ['Duke University'],
  'employers': ['University of Wisconsin–Madison'],
  'google_scholar_link': '',
  'notable_works': [''],
  'occupations': ['mathematician', 'university teacher'],
  'personal_websites': [],
  'wikidata_id': 'Q470503'}]

In [946]:
def get_author_info_v2(author_name):
    try:
        wiki_id = get_author_wikidata_id_new(author_name)
        return parse_wikidata_v2(wiki_id)
    except:
        return {}

In [947]:
get_author_info_v2('Anil K. Jain')

Q15783490


{'academic_degree': '',
 'age': 72,
 'alive': True,
 'awards': ['W. Wallace McDowell Award',
  'ACM Fellow',
  'Fellow of the International Association for Pattern Recognition',
  'IEEE Fellow',
  'Fellow of the SPIE',
  'AAAS Fellow'],
 'educated_at': ['Ohio State University',
  'Indian Institute of Technology Kanpur'],
 'employers': ['Michigan State University'],
 'google_scholar_link': 'https://scholar.google.com/citations?user=g-_ZXGsAAAAJ',
 'notable_works': [''],
 'occupations': ['computer scientist', 'engineer'],
 'personal_websites': [],
 'wikidata_id': 'Q15783490'}

In [433]:
entity = wikidata_client.get('Q21678689', load=False)

In [387]:
# Too slow
def get_field_of_works(entity):
    entries = entity.attributes['claims']['P101']
    ids = [i["mainsnak"]["datavalue"]["value"]["id"] for i in entries]
    values = []
    for qid in ids:
        d = wikidata_client.get(qid, load=True)
        values.append(d.data['labels']['en']['value'].capitalize())
    return values

In [466]:
def get_image(entity):
    image_prop = wikidata_client.get('P18')
    try:
        image = entity[image_prop].image_url
    except:
        image = None
    return image

def get_dob_and_age(entity):
    dob_prop = wikidata_client.get('P569')
    try:
        dob = entity[dob_prop]
        age = calculate_age(dob)
    except:
        dob = None
        age = None
    return dob, age

def get_dod_and_alive(entity):
    dod_prop = wikidata_client.get('P570')
    try:
        dod = entity[dod_prop]
        alive = False
    except:
        dod = None
        alive = True
    return dod, alive

def get_occupation(entity):
    occupation_prop = wikidata_client.get("P106")    
    try:
        occupation = entity[occupation_prop]
        occupation = wikidata_client.get(occupation.id, load=True).data['labels']['en']['value'].capitalize()
    except:
        occupation = None
    return occupation

def get_employer(entity):
    employer_prop = wikidata_client.get('P108')
    try:
        employer = entity[employer_prop]
        employer = wikidata_client.get(employer.id, load=True).data['labels']['en']['value'].capitalize()
    except:
        employer = None
    return employer

def get_webpage(entity):
    webpage_prop = wikidata_client.get('P856')
    try:
        webpage = entity[webpage_prop]
    except:
        webpage = None
    return webpage


def get_scholar(entity):
    google_scholar = wikidata_client.get('P1960')
    try:
        scholar = f"https://scholar.google.com/citations?user={entity[google_scholar]}" 
    except:
        scholar = None
    return scholar



In [937]:
k = wptools.page(wikibase='Q15783490', silent=True).get_wikidata()
wikidata_dict = k.data['wikidata']

In [938]:
try:
    if 'date of birth (P569)' in wikidata_dict and wikidata_dict['date of birth (P569)'] != '+2000-00-00T00:00:00Z':
        try:
            dob = wikidata_dict['date of birth (P569)']
#                 print(dob)
            if isinstance(dob, list):
                if len(dob) > 1:
                    dob = dob[0]
#                 print("dob erna:", dob)
            date_of_birth = datetime.strptime(dob.split("T")[0], '+%Y-%m-%d')
        except ValueError:
            date_of_birth = datetime.strptime(dob.split("T")[0][1:5], "%Y")
        age = calculate_age(date_of_birth)
    else:
        age = None
except:
    age = None
    
print(age)

72


In [866]:
import ast

In [849]:
new_authors = pd.read_csv("authors_with_affiliation_info.csv", index_col=0)

In [872]:
def get_author_affiliation_infos(author_id):
    info = new_authors[new_authors.id == int(author_id)]["info"].values[0]
    return ast.literal_eval(info)

In [873]:
%%timeit
get_author_affiliation_infos('100047651')

1000 loops, best of 3: 697 µs per loop


In [850]:
new_authors

,id,n_citation,n_pubs,name,pubs,tags,info
0,100047651,420,32,Florent Benaych-Georges,"[{'i': '2132657058', 'r': 0}, {'i': '276099004...","[{'t': 'Brownian motion'}, {'t': 'Convergence'...",{'additionalWebsite': 'http://en.wikipedia.org...
1,1000859103,259,5,Frederique Crete,"[{'i': '2079534654', 'r': 2}, {'i': '224933826...","[{'t': 'Psychophysics'}, {'t': 'Sound quality'...",{'gridPage': 'http://www.grid.ac/institutes/gr...
2,100110539,271,13,Maryam Karimzadehgan,"[{'i': '2546347449', 'r': 1}, {'i': '136907812...","[{'t': 'Machine learning'}, {'t': 'Web page'},...",{}
3,100167829,1184,117,Fabio Massimo Zanzotto,"[{'i': '371408518', 'r': 2}, {'i': '1487779154...","[{'t': 'Semantic data model'}, {'t': 'Cancer'}...",{'gridPage': 'http://www.grid.ac/institutes/gr...
4,100252078,1042,131,Stefano Serra-Capizzano,"[{'i': '2132492255', 'r': 1}, {'i': '252184247...","[{'t': 'Spectral power distribution'}, {'t': '...",{'gridPage': 'http://www.grid.ac/institutes/gr...
...,...,...,...,...,...,...,...
67803,2877861999,5716,27,J. Bobin,"[{'i': '2137276019', 'r': 18}, {'i': '20445647...",NaN,{'additionalWebsite': 'http://en.wikipedia.org...
67804,2877876007,1197,38,Tom A. M. Kevenaar,"[{'i': '2320012933', 'r': 3}, {'i': '148954825...",NaN,{'gridPage': 'http://www.grid.ac/institutes/gr...
67805,2878057644,630,7,N. Kannathal,"[{'i': '2132483662', 'r': 0}, {'i': '149475650...",NaN,{'gridPage': 'http://www.grid.ac/institutes/gr...
67806,287820616,1013,25,Stephan Bloehdorn,"[{'i': '2523573027', 'r': 0}, {'i': '193792410...","[{'t': 'Semantic technology'}, {'t': 'Metadata...","{'homepage': 'https://www.kit.edu/english/', '..."
